# <center> "Transition Effects Realizations" </center>

<div style="text-align: right"> 
        Gizem Gullu <br>
        2024, group number <br>
        230ADB075 <br>
</div>

### Course work theoretical part

<hr><b>Task description:</b><br><br>

{Description:

Develop an application in Python environment for creating animated transition effects between two images.
Realize 3 transition effects (Swipe, Curtain, Fade)
The user should be able to open two images using the user interface
In the app, a real-time animation for the transition of the selected two images must be displayed. No pre-recorded transition is acceptable. The only exception is if the video is rendered DURING runtime using the realized algorithms (no in-built functions) and then shown on the UI.
Do not use built-in methods functions or libraries to create transition effects (the transition effects must be programmed and you must be able to explain how they work)
Console programs (without user interface) will not be accepted.} 

    
<hr>

<hr>
<b>Theoretical Description of the Topic</b>
<br><br>

<b>Fade Transition Effect:</b><br>
It blends each picture gradually from the first to the second. I used linear interpolation of each pixel's values. For each frame, it calculates the next pixel value.
<br><br>

<b>Swipe Transition Effect:</b><br>
This replaces the first uploaded image with the second one row by row or column by column with frames. It shows a wiping effect. It keeps the remaining columns as they are in the first image and increments the width until the entire image is replaced with the second one. Key technique is that incrementing columns of pixels by copying.
<br><br>

<b>Curtain Transition:</b><br>
This calculates the center column of the image using the formula: <code>mid = image_width // 2</code>. It reveals pixels symmetrically from the center while keeping other columns unchanged. Main task is that it reveals pixels symmetrically outward. 
<br><br>

<hr>


<hr><b>Python libraries/packages:</b><br><br>
<pre>
<b>1. Tkinter:</b>   
    Tkinter is a easy to use Python toolkit. creating simple desktop applications is not complicated with this tool. I used it for creating GUI. 

<b>2. Pillow:</b>  
     For image processing, ı had choosen for Pillow library. It has some advantages like, easy manipulation of images, such as resizing, filtering, and saving in different formats. I used Pillow for loading, resizing, and displaying images.


<b>3. threading:</b>   
    Used to run the transition effects in the background, it enables running without any error with long procces. The main application thread remains responsive, allowing the user to interact with the interface during the transition of images.

<b>4. time:</b>    
    This module was used to show delays between each frame in GUI. This helped to control the animation speed and create for transitions without any problems. "Time" makes sure that each frame is displayed for an appropriate duration.
</pre>

<hr>


<hr><b>Information sources:</b><br><br>

{1) https://www.geeksforgeeks.org/python-gui-tkinter
 2) https://www.w3schools.com/python/numpy/numpy_intro.asp
 3) Some of the part I got help from OpenAi to make it more easy to understand code's structure
 4) https://neptune.ai/blog/image-processing-python
/}

    
<hr>

### Course work practical part

In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import time
import threading


 # creates app GU
class TransitionApp: 
    def __init__(self, root):
        self.root = root
        self.root.title("Image Transition Effects")
        self.root.configure(bg="#5BBDE1")  # adjust background 

    
        style = ttk.Style()
        style.configure("TButton", font=("Helvetica", 12, "bold"), padding=5, background="#E17F5B", foreground="white")
        style.map(
            "TButton",
            background=[("active", "#F2A288")],  
            foreground=[("active", "white")],
        )
        style.configure("TLabel", background="#5BBDE1", font=("Helvetica", 12, "bold"), foreground="#E17F5B")
        style.configure("TCombobox", font=("Helvetica", 12))


        self.canvas = tk.Canvas(root, width=500, height=500, bg="white", highlightthickness=0)
        self.canvas.pack(pady=10)

 
        btn_frame = tk.Frame(root, bg="#5BBDE1")
        btn_frame.pack(pady=5)
        ttk.Button(btn_frame, text="Upload Image 1", command=self.load_image1).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Upload Image 2", command=self.load_image2).pack(side=tk.LEFT, padx=10)

        # Selector for effects 
        ttk.Label(root, text="Select Transition Effect:").pack(pady=5)
        self.transition_type = tk.StringVar(value="Fade")
        self.transition_menu = ttk.Combobox(
            root, textvariable=self.transition_type, values=["Swipe", "Curtain", "Fade"], state="readonly"
        )
        self.transition_menu.pack()

        ttk.Button(root, text="Start Transition", command=self.start_transition).pack(pady=5)
        ttk.Button(root, text="Stop Transition", command=self.stop_transition).pack()

   
        self.progress = ttk.Progressbar(root, mode="determinate", length=300)
        self.progress.pack(pady=10)

        self.img1 = None
        self.img2 = None
        self.current_img_tk = None
        self.running = False

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def load_image1(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.img1 = Image.open(file_path).resize((500, 500))
            self.display_preview(self.img1)

    def load_image2(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.img2 = Image.open(file_path).resize((500, 500))

    def display_image(self, img):
        if not self.running:
            return
        self.current_img_tk = ImageTk.PhotoImage(img, master=self.root)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.current_img_tk)

    def display_preview(self, img):
        img_resized = img.resize((150, 150))
        preview_tk = ImageTk.PhotoImage(img_resized, master=self.root)
        self.canvas.create_image(10, 10, anchor=tk.NW, image=preview_tk)
        self.root.update()

    def clear_canvas(self):
        self.canvas.delete("all")
        self.current_img_tk = None

    def start_transition(self):
        if not self.img1 or not self.img2:
            print("Please upload both images!")
            return

        self.running = True
        self.progress["value"] = 0
        transition = self.transition_type.get()

        threading.Thread(target=self.run_transition, args=(transition,), daemon=True).start()

    def stop_transition(self):
        self.running = False
        self.clear_canvas()

    def run_transition(self, transition):
        if transition == "Fade":
            self.fade_transition()
        elif transition == "Swipe":
            self.swipe_transition()
        elif transition == "Curtain":
            self.curtain_transition()

    def fade_transition(self):
        img1_pixels = self.img1.load()
        img2_pixels = self.img2.load()
        blended_img = Image.new("RGB", (500, 500))
        blended_pixels = blended_img.load()

        for i, alpha in enumerate([x / 50 for x in range(51)]):  # 50 frames
            if not self.running:
                break
            for y in range(500):
                for x in range(500):
                    p1 = img1_pixels[x, y]
                    p2 = img2_pixels[x, y]
                    blended_pixels[x, y] = tuple(
                        int((1 - alpha) * c1 + alpha * c2) for c1, c2 in zip(p1, p2)
                    )
            self.display_image(blended_img)
            self.progress["value"] = (i + 1) * 2
            time.sleep(0.01)

    def swipe_transition(self):
        img1_pixels = self.img1.load()
        img2_pixels = self.img2.load()
        blended_img = Image.new("RGB", (500, 500))
        blended_pixels = blended_img.load()

        for i, x in enumerate(range(0, 500, 10)):  # Swipe 10 pixels 
            if not self.running:
                break
            for y in range(500):
                for col in range(x):
                    blended_pixels[col, y] = img2_pixels[col, y]
                for col in range(x, 500):
                    blended_pixels[col, y] = img1_pixels[col, y]
            self.display_image(blended_img)
            self.progress["value"] = (i + 1) * 2
            time.sleep(0.05)

    def curtain_transition(self):
        img1_pixels = self.img1.load()
        img2_pixels = self.img2.load()
        blended_img = Image.new("RGB", (500, 500))
        blended_pixels = blended_img.load()
        mid = 250

        for i, step in enumerate(range(0, mid, 10)):  # curtains for 10 pixels 
            if not self.running:
                break
            for y in range(500):
                for col in range(mid - step, mid + step):
                    if 0 <= col < 500:
                        blended_pixels[col, y] = img2_pixels[col, y]
                for col in range(0, mid - step):
                    blended_pixels[col, y] = img1_pixels[col, y]
                for col in range(mid + step, 500):
                    blended_pixels[col, y] = img1_pixels[col, y]
            self.display_image(blended_img)
            self.progress["value"] = (i + 1) * 2
            time.sleep(0.05)

    def on_close(self):
        self.stop_transition()
        self.root.destroy()


class EntranceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Fundemental of Image Processing Course")
        self.root.geometry("500x700")
        self.root.configure(bg="#5BBDE1")  

        content_frame = tk.Frame(root, bg="#5BBDE1")
        content_frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        description_label = tk.Label(
            content_frame,
            text=(
                "Create Transition Effects\n"
                "Between Images With 3 Different Options!"
            ),
            font=("Helvetica", 21, "bold"),
            bg="#5BBDE1",
            fg="#E17F5B",
            justify="center"
        )
        description_label.pack(pady=20)


        button_frame = tk.Frame(content_frame, bg="#5BBDE1")
        button_frame.pack(pady=20)

        # Start Button
        start_button = ttk.Button(
            button_frame,
            text="Start!",
            command=self.open_transition_window,
            style="Start.TButton",
        )
        start_button.pack(ipadx=20, ipady=5)

        # Button Styling
        style = ttk.Style()
        style.configure(
            "Start.TButton",
            font=("Helvetica", 14, "bold"),
            background="#E17F5B",  # Warm orange
            foreground="white",
            padding=10,
        )
        style.map(
            "Start.TButton",
            background=[("active", "#F2A288")],  
            foreground=[("active", "white")],
        )

    def open_transition_window(self):
        self.root.destroy()
        new_root = tk.Tk()
        TransitionApp(new_root)
        new_root.mainloop()


if __name__ == "__main__":
    entrance_root = tk.Tk()
    app = EntranceApp(entrance_root)
    entrance_root.mainloop()